# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

<Client: 'tcp://127.0.0.1:34883' processes=1 threads=4, memory=1.86 GiB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id    name         x         y
0  1010   Laura -0.349083 -0.820731
1  1018  George -0.803617 -0.705560
2  1025   Quinn  0.652247 -0.360885
3  1008   Wendy  0.979793  0.048558
4  1022   Quinn  0.769354  0.875786

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   1.000007e+03 -3.207876e-04  3.531596e-04
std    3.164607e+01  5.776059e-01  5.774679e-01
min    8.510000e+02 -9.999994e-01 -9.999996e-01
25%    9.790000e+02 -4.954270e-01 -4.919035e-01
50%    1.000000e+03  4.831714e-03  9.514034e-03
75%    1.022000e+03  5.043861e-01  5.053812e-01
max    1.161000e+03  9.999977e-01  9.999991e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
989,Bob,-0.2558986367418856
1005,Ray,-0.1955571864255303
1034,Quinn,0.8246409903498171
1034,Frank,0.04319374077888
1000,Wendy,0.162699516207657
1026,Patricia,-0.9585458902941596
1004,Hannah,-0.0986353537881907
1015,Ursula,0.9615134502018844
973,Norbert,0.2821330510708342


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id    name         x         y
0  1010   Laura -0.349083 -0.820731
1  1018  George -0.803617 -0.705560
2  1025   Quinn  0.652247 -0.360885
3  1008   Wendy  0.979793  0.048558
4  1022   Quinn  0.769354  0.875786

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id    name         x         y
0  1010   Laura -0.349083 -0.820731
1  1018  George -0.803617 -0.705560
2  1025   Quinn  0.652247 -0.360885
3  1008   Wendy  0.979793  0.048558
4  1022   Quinn  0.769354  0.875786

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id   name         x   y
0   989    Bob -0.255899 NaN
1  1005    Ray -0.195557 NaN
2  1034  Quinn  0.824641 NaN
3  1034  Frank  0.043194 NaN
4  1000  Wendy  0.162700 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-83ad0762-739f-4194-9c65-59f348cbe051')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id    name         x         y
0      1010   Laura -0.349083 -0.820731
1      1018  George -0.803617 -0.705560
2      1025   Quinn  0.652247 -0.360885
3      1008   Wendy  0.979793  0.048558
4      1022   Quinn  0.769354  0.875786
...     ...     ...       ...       ...
86395   984  Oliver -0.882975  0.783093
86396  1027  Ursula  0.442122 -0.029759
86397   984  Xavier  0.207410 -0.813991
86398   999  Ursula  0.740058 -0.006924
86399   955  George -0.135238 -0.988232

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-27.csv',
 'data/2000-01-15.csv',
 'data/2000-01-10.csv',
 'data/2000-01-01.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-21.csv',
 'data/2000-01-30.csv',
 'data/2000-01-29.csv',
 'data/2000-01-28.csv',
 'data/2000-01-20.csv',
 'data/2000-01-11.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv',
 'data/2000-01-08.csv',
 'data/2000-01-02.csv',
 'data/2000-01-04.csv',
 'data/2000-01-05.csv',
 'data/2000-01-17.csv',
 'data/2000-01-23.csv',
 'data/2000-01-26.csv',
 'data/2000-01-16.csv',
 'data/2000-01-24.csv',
 'data/2000-01-03.csv',
 'data/2000-01-06.csv',
 'data/2000-01-18.csv',
 'data/2000-01-12.csv',
 'data/2000-01-19.csv',
 'data/2000-01-25.csv',
 'data/2000-01-09.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-293ceeaa-0649-4587-b658-578e75c260e7'),
 Delayed('read_data-0dd7b77e-d829-444d-b6ba-ba5d0e1a47ae'),
 Delayed('read_data-6430d4bf-f2d1-4855-b816-23d2273d9cea'),
 Delayed('read_data-802ca993-d618-449d-9be8-bc85ea88648b'),
 Delayed('read_data-85d48ed9-d90b-423a-ad11-c90967af3ffe'),
 Delayed('read_data-bf4a6229-c7fd-4f98-9b75-1a46d54d8406'),
 Delayed('read_data-ed5a03b4-7ced-4f15-81b3-3c2c11f0a4d9'),
 Delayed('read_data-b62907d9-86b4-4c98-b7c9-32c6f13bde48'),
 Delayed('read_data-77c1e91e-1c3a-4d30-bc04-ab071debfb33'),
 Delayed('read_data-406aeae4-df2d-4a27-a416-2d2fbd4e95d2'),
 Delayed('read_data-613ebd01-c7b4-4ed8-8b56-7e2e4642765e'),
 Delayed('read_data-6da4b48e-5be1-4f57-912f-66a249b716dd'),
 Delayed('read_data-9c5b876f-604c-4e7d-808e-34fd745fe927'),
 Delayed('read_data-362884fc-b903-4066-b948-ccb9e7eabd53'),
 Delayed('read_data-b0f52fdb-9e51-4b6a-bf7d-dbbfce03fead'),
 Delayed('read_data-380bb0b5-1b4b-4018-ae93-a2ce9bea728b'),
 Delayed('read_data-64a01b16-3b81-4b1f-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id      name         x         y
0  1019   Norbert  0.690974  0.008318
1  1035  Patricia  0.653250  0.470658
2   972    Ingrid -0.865599  0.594963
3   984       Tim  0.146003  0.471499
4   948    Ursula  0.670260 -0.015103

In [26]:
df.describe().compute()

id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   1.000010e+03 -2.676304e-04  3.298364e-04
std    3.164298e+01  5.776341e-01  5.774504e-01
min    8.510000e+02 -9.999994e-01 -9.999996e-01
25%    9.790000e+02 -4.954270e-01 -4.919035e-01
50%    1.000000e+03  4.831714e-03  9.514034e-03
75%    1.022000e+03  5.043861e-01  5.053812e-01
max    1.161000e+03  9.999977e-01  9.999991e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.